In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [2]:
INPUT_DIR = "../graphfleet/output/20240730-073603/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

[2024-07-31T11:33:40Z WARN  lance::dataset] No existing dataset at /Volumes/Samsung-SSD-T7/Qredence/GraphFleet/GraphFleet/notebooks/../graphfleet/output/20240730-073603/artifacts/lancedb/entity_description_embeddings.lance, it will be created


Entity count: 3018


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,"""MICROSOFT RESEARCH""","""ORGANIZATION""","""Microsoft Research is a division of Microsoft...",e7e620f804861b86c33f80a0f61ebb8c,20,1,0,b45241d70f0e43fca764df95b2b81f77,1.0,"[-0.008307618089020252, -0.04216441884636879, ...",None,b45241d70f0e43fca764df95b2b81f77,16.441730,0.053836
1,0,"""MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES""","""ORGANIZATION""","""Microsoft Strategic Missions and Technologies...",e7e620f804861b86c33f80a0f61ebb8c,20,1,1,4119fd06010c494caa07f439b333f4c5,1.0,"[-0.007349406369030476, -0.05208325758576393, ...",None,4119fd06010c494caa07f439b333f4c5,16.237318,1.418194
2,0,"""MICROSOFT OFFICE OF THE CTO""","""ORGANIZATION""","""Microsoft Office of the CTO is a division wit...",e7e620f804861b86c33f80a0f61ebb8c,20,1,2,d3835bf3dda84ead99deadbeac5d0d7d,1.0,"[-0.0090823695063591, -0.039205294102430344, 0...",None,d3835bf3dda84ead99deadbeac5d0d7d,17.169176,-1.057973
3,0,"""GRAPH RAG""","""SUBDOMAIN""",Graph RAG is an advanced retrieval-augmented g...,"0274e77e2fcec8973c9768c464c6e82d,08e6ee9b2e040...",20,41,3,077d2820ae1845bcbb1803379a3d1eae,41.0,"[-0.042907144874334335, -0.12994633615016937, ...",None,077d2820ae1845bcbb1803379a3d1eae,19.181860,-12.728170
4,0,"""RETRIEVAL-AUGMENTED GENERATION (RAG)""","""SUBDOMAIN""",RETRIEVAL-AUGMENTED GENERATION (RAG) is a meth...,"55ec70780a07388aca4be80802ea19f1,e7e620f804861...",20,2,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,2.0,"[-0.012449133209884167, -0.05482371896505356, ...",None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,11.581391,4.506033


#### Read relationships

In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 831


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,"""MICROSOFT RESEARCH""","""GRAPH RAG""",1.0,"""Microsoft Research is involved in developing ...",[e7e620f804861b86c33f80a0f61ebb8c],d21a1fef903f4a399bd3cd366aad3c9e,0,1,41,42
1,"""MICROSOFT STRATEGIC MISSIONS AND TECHNOLOGIES""","""GRAPH RAG""",1.0,"""Microsoft Strategic Missions and Technologies...",[e7e620f804861b86c33f80a0f61ebb8c],fc596a598ff74a4c843e405b597551b5,1,1,41,42
2,"""MICROSOFT OFFICE OF THE CTO""","""GRAPH RAG""",1.0,"""Microsoft Office of the CTO supports the tech...",[e7e620f804861b86c33f80a0f61ebb8c],e2aacff6b4404574b818e7a3ece57b5b,2,1,41,42
3,"""GRAPH RAG""","""RETRIEVAL-AUGMENTED GENERATION (RAG)""",1.0,"""Graph RAG builds upon the principles of RAG t...",[e7e620f804861b86c33f80a0f61ebb8c],2ec5cae98c7a485881f0680fbca6d67f,3,41,2,43
4,"""GRAPH RAG""","""QUERY-FOCUSED SUMMARIZATION (QFS)""",1.0,"""Graph RAG integrates QFS to provide comprehen...",[e7e620f804861b86c33f80a0f61ebb8c],c87b815d61af448596d3194a804b57b3,4,41,2,43


In [5]:
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Claim records: 619


#### Read community reports

In [6]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 107


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,102,# LLM and Prompt Engineering Techniques\n\nThe...,2,8.5,LLM and Prompt Engineering Techniques,The impact severity rating is high due to the ...,The community revolves around Large Language M...,[{'explanation': 'LLM (Large Language Model) i...,"{\n ""title"": ""LLM and Prompt Engineering Te...",4c32e8a9-3c66-4884-b2a2-40af5a2931b3
1,103,# RLPrompt and Rephrase and Respond\n\nThe com...,2,7.5,RLPrompt and Rephrase and Respond,The impact severity rating is high due to the ...,The community is centered around two key subdo...,[{'explanation': 'RLPrompt is a subdomain dedi...,"{\n ""title"": ""RLPrompt and Rephrase and Res...",8a4859d6-dcfc-45bc-b5ce-2d13bad46a5b
2,104,# Answer Engineering and Related Concepts\n\nT...,2,7.5,Answer Engineering and Related Concepts,The impact severity rating is high due to the ...,The community revolves around the concept of A...,[{'explanation': 'Answer Engineering is the ce...,"{\n ""title"": ""Answer Engineering and Relate...",b1127185-f950-4b89-8425-bd05c6233c0b
3,105,# Evaluation Techniques and Methods in Prompti...,2,8.5,Evaluation Techniques and Methods in Prompting,The impact severity rating is high due to the ...,The community revolves around the concept of '...,[{'explanation': 'Evaluation is the central en...,"{\n ""title"": ""Evaluation Techniques and Met...",3376c5d8-0b11-471e-8ca1-e1da35ca16ee
4,106,# Chain-of-Thought Prompting and Prompting Tec...,2,8.5,Chain-of-Thought Prompting and Prompting Techn...,The impact severity rating is high due to the ...,The community revolves around the 'Chain-of-Th...,[{'explanation': 'Chain-of-Thought Prompting i...,"{\n ""title"": ""Chain-of-Thought Prompting an...",b463b52d-8749-484f-9e58-8aa581a53887


#### Read text units

In [7]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 90


,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,e7e620f804861b86c33f80a0f61ebb8c,From Local to Global: A Graph RAG Approach to\...,1000,[0668cddc5f873265ba50da5a0a06edad],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[d21a1fef903f4a399bd3cd366aad3c9e, fc596a598ff...","[dd506afa-5652-4f10-bcfd-4c1a53ad830e, db4cc31..."
1,55ec70780a07388aca4be80802ea19f1,2006b) by asking questions of a global nature....,1000,[0668cddc5f873265ba50da5a0a06edad],"[077d2820ae1845bcbb1803379a3d1eae, 3671ea0dd4e...","[64be9b98299f4d349e0f4358685ca235, 8302a03f6ed...",[286a41a7-f5a6-434c-a898-955e99181484]
2,1e97679db415c7c17b35542305f23ced,\nfavorable performance over source text summa...,1000,[0668cddc5f873265ba50da5a0a06edad],"[077d2820ae1845bcbb1803379a3d1eae, c9632a35146...","[64be9b98299f4d349e0f4358685ca235, 8302a03f6ed...","[bc762d07-ba16-491c-aec7-af64b17cc3b0, e7683f1..."
3,c5a27b7f9fad18a6ad22416c453ae383,is resilient to such variations provided ther...,1000,[0668cddc5f873265ba50da5a0a06edad],"[077d2820ae1845bcbb1803379a3d1eae, f7e11b0e297...","[64be9b98299f4d349e0f4358685ca235, 8302a03f6ed...",[2cc72bca-6198-44e7-9ee0-55048ec88532]
4,da636ab056625c618d1656cfc725630c,user query is generated as follows:\n•Prepare...,1000,[0668cddc5f873265ba50da5a0a06edad],"[077d2820ae1845bcbb1803379a3d1eae, 1fd3fa8bb5a...","[64be9b98299f4d349e0f4358685ca235, 8302a03f6ed...","[6b015579-5706-4481-bb3d-745ff05b3e70, 7c635fb..."


In [12]:
from dotenv import load_dotenv
import os
import tiktoken
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType # Replace with actual module names

# Load environment variables from a .env file
load_dotenv()
api_key = os.getenv('GRAPHRAG_API_KEY')
llm_model = os.getenv('GRAPHRAG_LLM_MODEL')
embedding_model = os.getenv('GRAPHRAG_EMBEDDING_MODEL')

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

Failed to create OpenAI client {'ChatOpenAI': 'api_base is required for Azure OpenAI'}


ValueError: api_base is required for Azure OpenAI

### Create local search context builder

In [9]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

NameError: name 'covariates' is not defined

### Create local search engine

In [ ]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [ ]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [ ]:
result = await search_engine.asearch("Tell me about Agent Mercer")
print(result.response)

In [ ]:
question = "Tell me about Dr. Jordan Hayes"
result = await search_engine.asearch(question)
print(result.response)

#### Inspecting the context data used to generate the response

In [ ]:
result.context_data["entities"].head()

In [ ]:
result.context_data["relationships"].head()

In [ ]:
result.context_data["reports"].head()

In [ ]:
result.context_data["sources"].head()

In [ ]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [ ]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [ ]:
question_history = [
    "Tell me about Agent Mercer",
    "What happens in Dulce military base?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)